In [30]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf

physical_devices= tf.config.list_physical_devices()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
import numpy as np
from sklearn.model_selection import train_test_split
from loader.CSVLoader import csv_loader

gt,wave = csv_loader()
all_data = np.load("./experiment1/exp_1d.npy")
test_data = np.load('./experiment1/exp_1d_test.npy')
all_data.shape

(1732, 150)

In [39]:
from creators.CNN3DCreator import  CNN3DNetworkParams
from creators.ModelParams import Conv1DParams, Conv2DParams, Conv3DParams, DeepLayerParams, FlattenParams, GenericNeuralNetworkParams
from creators.CNN1DCreator import  CNN1DNetworkParams
from creators.CNN2DCreator import CNN2DNetworkParams
from prep.HelperBlocks import pipe
import tensorflow as tf

drop=0.53
epochs=1
optimizer = tf.keras.optimizers.Adam
folds = 5


params_cnn3d = CNN3DNetworkParams(
    conv=[Conv3DParams(kernel=(10,10,1),pool_size=(10,10,1),filters=32,normalization=True,activation="relu", dropout=drop),
          Conv3DParams(kernel=(1,1,15),pool_size=(1,1,15),filters=128, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: np.expand_dims(d, axis=-1),
        ]),
        epochs=epochs,
        optimizer=optimizer
        )
)

params_cnn2d = CNN2DNetworkParams(
    conv=[Conv2DParams(kernel=(5,5),pool_size=(2,2),filters=64,normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(3,3),pool_size=(2,2),filters=512, normalization=True,activation="relu", dropout=drop),
          Conv2DParams(kernel=(2,2),pool_size=(2,2),filters=512, normalization=True,activation="relu", dropout=drop),
          ],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: data,
        epochs=epochs,
        optimizer=optimizer
        )
)

params_cnn1d = CNN1DNetworkParams(
    conv=[Conv1DParams(kernel=10,pool_size=5,filters=500,normalization=True,activation="relu", dropout=drop),
          Conv1DParams(kernel=5,pool_size=5,filters=1500, normalization=True,activation="relu", dropout=drop)
          ],
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
          flatten=FlattenParams(dropout=drop,normalization=True),
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[  
            lambda d: np.expand_dims(d, axis=-1),
        ]),
        epochs=epochs,
        optimizer=optimizer
        )
)

In [40]:
from creators.ModelParams import ConvolutionalRecurrentLayerParams, RandomForestParams, RecurrentLayerParams, SVRParams
from creators.CRNNCreator import CRNNNetworkParams
from creators.RNNCreator import RNNNetworkParams
from creators.CLSTMCreator import CLSTMNetworkParams
from prep.HelperBlocks import split_3d_to_timeseries

params_crnn = CRNNNetworkParams(
    conv=[Conv3DParams(kernel=(1,10,10),pool_size=(1,10,10),filters=50,normalization=True,dropout=drop)],
    rec=[RecurrentLayerParams(type=tf.keras.layers.LSTM,units=125,bidirectional=True,dropout=drop),
         RecurrentLayerParams(type=tf.keras.layers.LSTM,units=125,bidirectional=True,dropout=drop)],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: pipe(data,[
            lambda d: split_3d_to_timeseries(d,15)
        ]),
        epochs=epochs,
        optimizer=optimizer
        )
)

params_lstm = RNNNetworkParams(
    rec=[
         RecurrentLayerParams(type=tf.keras.layers.LSTM,units=128,bidirectional=True,dropout=drop,normalization=True)],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test:pipe(data,[
            lambda d: np.reshape(d, (d.shape[0],15,10)),
        ]),
        epochs=epochs,
        optimizer=optimizer
    ))

params_clstm = CLSTMNetworkParams(
    rec=[ConvolutionalRecurrentLayerParams(filters=15,kernel_size=(3,3))],
    flatten=FlattenParams(dropout=drop,normalization=True),
    deep=[DeepLayerParams(neurons=1000,dropout=drop,normalization=True),
          DeepLayerParams(neurons=400,dropout=drop,normalization=True)],
    generic=GenericNeuralNetworkParams(
        prepare_function=lambda data, is_test: data.transpose((0,3,1,2)).reshape((
            data.shape[0],
            15,
            10,
            data.shape[1],
            data.shape[2])).transpose((0,2,3,4,1)),
        epochs=epochs,
        optimizer=optimizer
        )
)

params_svr = SVRParams(lambda data, is_test: data)
params_rf = RandomForestParams(lambda data, is_test: data)

In [41]:
from ModelExperiment import ModelExperiment

experiment = ModelExperiment()
experiment.run_experiment(
    params_vector=[params_cnn1d,params_lstm],
    data=[all_data,all_data,all_data,all_data],
    gt=[gt["pH"].values,gt["pH"].values,gt["pH"].values,gt["pH"].values],
    folds=folds,
    test_data=[test_data,test_data,test_data,test_data]
)

37/37 [==============================] - 1s 23ms/step


In [44]:
experiment.models[0].save_history('./history1.csv')

In [ ]:
experiment.get_errors_report('./errors1.csv')

In [ ]:
print(experiment.models[1].last_crossval_predict_time)

0.04428415298461914


In [ ]:
print(experiment.models[1].last_crossval_fit_time)

42.80209965705872


In [ ]:
print(experiment.models[1].crossval_rs
)

[0.07870845 0.07444822 0.08654511 0.078417   0.02610544]
